```{currentmodule} optimap
```

In [ ]:
from optimap.utils import jupyter_render_animation as render

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/mask.ipynb>`, or as a {download}`python script <converted/mask.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial.
```

# Tutorial 3: Masking / Segmentation

This tutorial explains how to select a region of interest in a video using ``optimap``. For instance, it is possible to use ``optimap`` to automatically select the heart surface, to manually draw a region of interest in the video image, or to ignore parts of the video close to the boundary. The post-processing can then be applied to  the masked or segmented part of the video image, for instance. ``optimap`` provides several easy-to-use routines for these purposes.

We will first load an example video file using the following code:

In [ ]:
import optimap as om
import numpy as np
import matplotlib.pyplot as plt

filepath = om.download_example_data('VF_Rabbit_1_warped.npy')
video_warped = om.load_video(filepath)
om.print_properties(video_warped)

frame = video_warped[0]  # first frame

## Automatic Background Segmentation

We can then use ``optimap``'s  {func}`background_mask` function to automatically separate much brighter tissue from the dark background:

In [ ]:
background_mask = om.background_mask(frame, title='Automatic background mask')
om.save_mask('background_mask.png', background_mask)

The {func}`background_mask` function creates a two-dimensional binary array with ``True`` for background (here shown in red) and ``False`` for tissue. Here, we used the first frame of the video to create the mask. {func}`background_mask` and {func}`foreground_mask` automatically estimate an optimal threshold for the separation of foreground and background using the GHT {cite}`Barron2020` algorithm (see {func}`image.detect_background_threshold`). Pixels with a value below this threshold (here ``401.0``) are considered background, and pixels with a value above this threshold are considered foreground. The threshold can be adjusted manually, if desired:

In [ ]:
foreground_mask = om.foreground_mask(frame, threshold=500, title='Manual threshold foreground mask')

Here we specified a threshold value of ``500`` to generate a foreground mask. Note that both functions only separate foreground from background, i.e. they do not distinguish the ventricles from the atria. For this purpose, we need to use a different approach.

## Manual Segmentation using Drawing Tool

We can manually draw a mask and select a region (e.g. the atria) using ``optimap``'s mask drawing tool:

In [ ]:
manual_mask = om.interactive_mask(frame, initial_mask=foreground_mask)

The drawing tool can be used to draw one or several arbitrary regions and the tool automatically creates the corresponding binary array as a mask. A drawn mask can be inverted or deleted, edits can be undone or redone, etc. (see Documentation). Simply use the above function and a window will pop up:

<img align="center" width="400" src="https://cardiacvision.ucsf.edu/sites/g/files/tkssra6821/f/optimap%20tutorials%20mask%201.png">

<img align="center" width="400" src="https://cardiacvision.ucsf.edu/sites/g/files/tkssra6821/f/optimap%20tutorials%20mask%202.png">

The following table lists the available keybindings:
| Key/Mouse           | Action                    |
|---------------------|---------------------------|
| `Scroll`            | Zoom in/out               |
| `ctrl+z` or `cmd+z` | Undo                      |
| `ctrl+y` or `cmd+y` | Redo                      |
| `v`                 | Toggle mask visibility    |
| `d`                 | Draw/Lasso mode           |
| `e`                 | Erase mode                |
| `q`                 | Quit                      |

To edit a mask in different program, e.g. [GIMP](https://gimp.org), save the mask as a PNG file:
```python
om.save_mask("mask.png", background_mask, image=frame)
```
The ``image`` argument is optional, but if provided, the mask will be saved as the alpha channel of the image. For editing the mask in GIMP select the alpha channel and use the paintbrush tool to edit the mask. The mask can then be loaded back into ``optimap`` using the following code:
```python
background_mask = om.load_mask("mask.png")
```

The {func}`save_mask` and {func}`load_mask` functions support a variety of file formats, including PNG, TIFF, and NPY. See the documentation for more information.

To visualize the mask, use {func}`show_mask`:

In [ ]:
fn = om.download_example_data('VF_Rabbit_1_warped_mask.npy', silent=True)
manual_mask = om.load_mask(fn)

In [ ]:
om.show_mask(manual_mask, image=frame, title='Manual mask');

## Refining the mask

Not in all cases the mask is perfect, and it may be necessary to adjust the mask. The automatic thresholding may not work well for all videos, and the mask may need to be adjusted. For instance, the mask may be too large or too small, or it may contain holes. We can use {func}`image.erode_mask` to shrink the mask by 10 pixels, or {func}`image.dilate_mask` to expand the mask by 10 pixels:

In [ ]:
mask = np.logical_not(manual_mask)

fig, axs = plt.subplots(1, 3)
dilated = om.image.dilate_mask(mask, iterations=10, show=False)
eroded = om.image.erode_mask(mask, iterations=10, border_value=True, show=False)

om.show_mask(mask, image=frame, ax=axs[0], title='Original')
om.show_mask(eroded, image=frame, ax=axs[1], title='Eroded')
om.show_mask(dilated, image=frame, ax=axs[2], title='Dilated')
plt.tight_layout()
plt.show()

Use {func}`image.fill_mask` to fill holes in the mask, to keep only the largest connected component (island) of the mask use {func}`image.largest_mask_component`. The {func}`image.largest_mask_component` function also has a ``invert`` argument to invert the mask before selecting the largest component (e.g. to keep the largest hole in the mask when working with background masks).

In [ ]:
mask = om.foreground_mask(frame, threshold=1800, show=False)

fig, axs = plt.subplots(1, 3)
om.show_mask(mask, image=frame, title='Original', ax=axs[0])
om.image.fill_mask(mask, image=frame, title='fill_mask()', ax=axs[1])
om.image.largest_mask_component(mask, image=frame, title='largest_mask_component()', ax=axs[2])
plt.tight_layout()
plt.show()

The opening and closing morphological operations combine erosion and dilation. For instance, {func}`image.open_mask` shrinks the mask and then expands it, while {func}`image.close_mask` expands the mask by and then shrinks it. By doing several iterations of this it can be used to remove small islands, fill holes, or to smooth the mask. See the [scipy documentation](https://docs.scipy.org/doc/scipy/tutorial/ndimage.html#morphology) for more information on morphological operations.

In [ ]:
mask = om.foreground_mask(frame, threshold=1400, show=False)

fig, axs = plt.subplots(1, 3)
om.show_mask(mask, image=frame, title='Original', ax=axs[0])
om.image.open_mask(mask, iterations=10, image=frame, ax=axs[1], title='open_mask()')
om.image.close_mask(mask, iterations=10, image=frame, ax=axs[2], title='close_mask()')
plt.tight_layout()
plt.show()

## Working with masks

```{warning}
This tutorial is currently under development. We will add more information soon.
```

In [ ]:
mask = om.image.dilate_mask(manual_mask, iterations=2, image=frame, show=False)

video_warped = video_warped.astype(np.float32)
video_warped[:, mask] = np.nan

In [ ]:
om.show_video(video_warped)

In [ ]:
render(lambda: om.show_video(video_warped))